In [1]:
""" My first attempt looking at this data in the context of graphical models """

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import pgmpy

import warnings
warnings.filterwarnings("ignore")

sns.set(style="ticks")
sns.set_context(context="talk")

In [2]:
infile = "/Users/kmcmanus/Documents/classes/digitalhealth_project/data/formatted_data/20200628_sleep_pos_5S_cleaned.csv"
df = pd.read_csv(infile, index_col='datetime', parse_dates=True, infer_datetime_format=True)
df["sleep_night"] = pd.to_datetime(df["sleep_night"])
df = df[["ODI", "orient_bin", "Pulse Rate(bpm)", "hour"]]
df = df.dropna()
#df = df[df["hour"] < 5]
#df.loc[df["Pulse Rate(bpm)"] > 70, "Pulse Rate(bpm)"] = 70
#df.loc[df["Pulse Rate(bpm)"] < 70, "Pulse Rate(bpm)"] = 60
df.loc[df["hour"] < 12, "hour"] = 0
df.loc[df["hour"] > 12, "hour"] = 1
df["pulse_bin"] = 1
df.loc[(df["pulse_bin"] > 80), "pulse_bin"] = 2
df.loc[(df["pulse_bin"] < 60), "pulse_bin"] = 0
df = df[["ODI", "orient_bin", "pulse_bin", "hour"]]
print("Total # rows: {}".format(df.shape[0]))
df.head()

Total # rows: 100818


,ODI,orient_bin,pulse_bin,hour
datetime,,,,
2020-04-21 20:59:00,0.0,1.0,0,1
2020-04-21 20:59:05,0.0,1.0,0,1
2020-04-21 20:59:10,0.0,1.0,0,1
2020-04-21 20:59:15,0.0,1.0,0,1
2020-04-21 20:59:20,0.0,1.0,0,1


In [3]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.estimators import BayesianEstimator


# orient_bin -> ODI
# hour -> ODI
# Pulse Rate -> ODI
model = BayesianModel([('orient_bin', 'ODI'), ('hour', 'ODI'), ('pulse_bin', 'ODI')]) 

from pgmpy.estimators import ParameterEstimator # Literally had to go in an manually change this file
from pgmpy.estimators import BDeuScore 
pe = ParameterEstimator(model, df)
print("\n", pe.state_counts('orient_bin'))  # unconditional
print("\n", pe.state_counts('hour'))  # unconditional
print("\n", pe.state_counts('pulse_bin'))  # unconditional
print("\n", pe.state_counts('ODI'))  # conditional on orient_bin, hour, pulse rate


       orient_bin
-1.0       28653
 0.0       29534
 1.0       42631

     hour
0  62687
1  38131

    pulse_bin
0     100818

 hour            0                   1             
orient_bin   -1.0    0.0    1.0  -1.0   0.0    1.0
pulse_bin       0      0      0     0     0      0
ODI                                               
0.0         19647  20206  21974  8831  8565  20425
1.0           100    675     85    75    88    147


In [4]:
pe.state_names['hour']

[0, 1]

In [5]:
#model.fit(df, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
model.fit(df, estimator=BayesianEstimator, prior_type="K2")
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(type(cpd))
    print(cpd)

CPD of orient_bin:
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
+------------------+----------+
| orient_bin(-1.0) | 0.284207 |
+------------------+----------+
| orient_bin(0.0)  | 0.292945 |
+------------------+----------+
| orient_bin(1.0)  | 0.422848 |
+------------------+----------+
CPD of ODI:
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
+------------+----------------------+----------------------+----------------------+----------------------+---------------------+----------------------+
| hour       | hour(0)              | hour(0)              | hour(0)              | hour(1)              | hour(1)             | hour(1)              |
+------------+----------------------+----------------------+----------------------+----------------------+---------------------+----------------------+
| orient_bin | orient_bin(-1.0)     | orient_bin(0.0)      | orient_bin(1.0)      | orient_bin(-1.0)     | orient_bin(0.0)     | orient_bin(1.0)      |
+------------+----------------------+----

In [6]:
# Ok what do I do with this?
# I can use a graphical model to improve accuracy, once I have the features
#ok = model.get_cpds(node=["ODI", "Pulse Rate(bpm)", "hour", "orient_bin"])
ok = model.get_cpds(node="ODI")

In [7]:
#print(ok)
print(ok.variables)
print(ok.get_cardinality(variables=["ODI", "pulse_bin", "hour", "orient_bin"]))
#print(ok.maximize(["ODI"]))
okf = ok.to_factor()
#okf_max = okf.maximize(["hour"])
#print(okf_max)
print(okf.values)
#print(ok)
print(okf)
#print(okf.values[1]) # ODI x PulseRate x hour x orient_bin
from numpy import unravel_index
max_loc = unravel_index(np.argmax(okf.values[1]), okf.values[1].shape)
#okf.values[1][1][4][2] # ODI=1, PulseRate=2nd lowest, hour=4th hour, orient_bin=2nd bin
#print(np.argmax(okf.values[1]))

min_loc = unravel_index(np.argmin(okf.values[1]), okf.values[1].shape)
print(max_loc)
print(min_loc)

['ODI', 'hour', 'orient_bin', 'pulse_bin']
{'ODI': 2, 'pulse_bin': 1, 'hour': 2, 'orient_bin': 3}
[[[[0.99488582]
   [0.96762917]
   [0.99610172]]

  [[0.99146834]
   [0.98971693]
   [0.99280645]]]


 [[[0.00511418]
   [0.03237083]
   [0.00389828]]

  [[0.00853166]
   [0.01028307]
   [0.00719355]]]]
+----------+---------+------------------+--------------+--------------------------------------+
| ODI      | hour    | orient_bin       | pulse_bin    |   phi(ODI,hour,orient_bin,pulse_bin) |
+==========+=========+==================+==============+======================================+
| ODI(0.0) | hour(0) | orient_bin(-1.0) | pulse_bin(0) |                               0.9949 |
+----------+---------+------------------+--------------+--------------------------------------+
| ODI(0.0) | hour(0) | orient_bin(0.0)  | pulse_bin(0) |                               0.9676 |
+----------+---------+------------------+--------------+--------------------------------------+
| ODI(0.0) | hour(0) | orie

In [8]:
print("For max prob")
#print("value {}".format(okf.values[1][3][1][2]))
#print("value {}".format(okf.values[2][1][2][3]))
print("Pulse : {}".format(pe.state_names['hour'][max_loc[0]]))
print("hour : {}".format(pe.state_names['orient_bin'][max_loc[1]]))
print("orient_bin : {}".format(pe.state_names['pulse_bin'][max_loc[2]]))

For max prob
Pulse : 0
hour : 0.0
orient_bin : 0


In [9]:
print("For min prob")
#print("value {}".format(okf.values[1][6][0][2]))
print("Pulse : {}".format(pe.state_names['hour'][min_loc[0]]))
print("hour : {}".format(pe.state_names['orient_bin'][min_loc[1]]))
print("orient_bin : {}".format(pe.state_names['pulse_bin'][min_loc[2]]))

For min prob
Pulse : 0
hour : 1.0
orient_bin : 0


In [10]:
from pgmpy.estimators import ExhaustiveSearch
from pgmpy.estimators import BDeuScore, K2Score, BicScore

bic = BicScore(df)

es = ExhaustiveSearch(df, scoring_method=bic)
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())

[('ODI', 'hour'), ('ODI', 'orient_bin'), ('hour', 'orient_bin')]

All DAGs by score:
-180172.63086701918 [('ODI', 'orient_bin'), ('hour', 'orient_bin'), ('hour', 'ODI'), ('pulse_bin', 'ODI'), ('pulse_bin', 'hour'), ('pulse_bin', 'orient_bin')]
-180172.63086701918 [('ODI', 'orient_bin'), ('hour', 'orient_bin'), ('hour', 'pulse_bin'), ('hour', 'ODI'), ('pulse_bin', 'ODI'), ('pulse_bin', 'orient_bin')]
-180172.63086701918 [('ODI', 'orient_bin'), ('ODI', 'pulse_bin'), ('hour', 'orient_bin'), ('hour', 'pulse_bin'), ('hour', 'ODI'), ('pulse_bin', 'orient_bin')]
-180172.63086701918 [('ODI', 'orient_bin'), ('ODI', 'pulse_bin'), ('hour', 'orient_bin'), ('hour', 'pulse_bin'), ('hour', 'ODI'), ('orient_bin', 'pulse_bin')]
-180172.63086701918 [('ODI', 'hour'), ('ODI', 'orient_bin'), ('hour', 'orient_bin'), ('pulse_bin', 'ODI'), ('pulse_bin', 'hour'), ('pulse_bin', 'orient_bin')]
-180172.63086701918 [('ODI', 'hour'), ('ODI', 'orient_bin'), ('ODI', 'pulse_bin'), ('hour', 'orient_bin'), ('pulse_bin',

In [11]:
# I want to find the highest probability combination, and compare that to the lowest
# probability combination

# Just get the matrix and identify the highest

#model.get_cpds(node='ODI')
#model.maximize

AttributeError: 'BayesianModel' object has no attribute 'maximize'